Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model with ONNX Runtime on CPU

In this tutorial, you'll be introduced to how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

This notebook is for CPU inference. For GPU inferenece, please look at another notebook [Inference PyTorch Bert Model with ONNX Runtime on GPU](PyTorch_Bert-Squad_OnnxRuntime_GPU.ipynb).

## 0. Prerequisites ##

If you have Jupyter Notebook, you may directly run this notebook. We will use pip to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/) and other required packages.

Otherwise, you can setup a new environment. First, we install [AnaConda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.6
conda activate cpu_env
conda install pytorch torchvision cpuonly -c pytorch
pip install onnxruntime
conda install jupyter
jupyter notebook
```
The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

In [1]:
# Install or upgrade PyTorch 1.4.0 and OnnxRuntime for CPU-only.
import sys
!{sys.executable} -m pip install --upgrade torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install --upgrade onnxruntime

# Install other packages used in this notebook.
!{sys.executable} -m pip install transformers==2.5.1
!{sys.executable} -m pip install wget psutil onnx pytz pandas py-cpuinfo py3nvml netron

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.4.0+cpu in d:\anaconda3\envs\cpu_env\lib\site-packages (1.4.0+cpu)
Requirement already up-to-date: torchvision==0.5.0+cpu in d:\anaconda3\envs\cpu_env\lib\site-packages (0.5.0+cpu)
Requirement already up-to-date: onnxruntime in d:\anaconda3\envs\cpu_env\lib\site-packages (1.2.0)


## 1. Load Pretrained Bert model ##

We begin by downloading the SQuAD data file and store them in the specified location.

In [2]:
import os

cache_dir = "./squad"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

Specify some model configuration variables and constant.

In [3]:
# For fine tuned large model, the model name is "bert-large-uncased-whole-word-masking-finetuned-squad". Here we use bert-base for demo.
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64

# Enable overwrite to export onnx model and download latest script each time when running this notebook.
enable_overwrite = True

# Total samples to inference. It shall be large enough to get stable latency measurement.
total_samples = 100

Start to load model from pretrained. This step could take a few minutes. 

In [4]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:total_samples], # convert just enough examples for this notebook
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

add example index and unique id: 100%|███████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [5]:
output_dir = "./onnx"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)   
export_model_path = os.path.join(output_dir, 'bert-base-cased-squad.onnx')

import torch
device = torch.device("cpu")

# Get the first example data to run the model and export it to ONNX
data = dataset[0]
inputs = {
    'input_ids':      data[0].to(device).reshape(1, max_seq_length),
    'attention_mask': data[1].to(device).reshape(1, max_seq_length),
    'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
}

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
model.to(device)

if enable_overwrite or not os.path.exists(export_model_path):
    with torch.no_grad():
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                          args=tuple(inputs.values()),                      # model input (or a tuple for multiple inputs)
                          f=export_model_path,                              # where to save the model (can be a file or file-like object)
                          opset_version=11,                                 # the ONNX version to export the model to
                          do_constant_folding=True,                         # whether to execute constant folding for optimization
                          input_names=['input_ids',                         # the model's input names
                                       'input_mask', 
                                       'segment_ids'],
                          output_names=['start', 'end'],                    # the model's output names
                          dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                        'input_mask' : symbolic_names,
                                        'segment_ids' : symbolic_names,
                                        'start' : symbolic_names,
                                        'end' : symbolic_names})
        print("Model exported at ", export_model_path)

Model exported at  ./onnx\bert-base-cased-squad.onnx


## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [6]:
import time

# Measure the latency. It is not accurate using Jupyter Notebook, it is recommended to use standalone python script.
latency = []
with torch.no_grad():
    for i in range(total_samples):
        data = dataset[i]
        inputs = {
            'input_ids':      data[0].to(device).reshape(1, max_seq_length),
            'attention_mask': data[1].to(device).reshape(1, max_seq_length),
            'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
        }
        start = time.time()
        outputs = model(**inputs)
        latency.append(time.time() - start)
print("PyTorch {} Inference time = {} ms".format(device.type, format(sum(latency) * 1000 / len(latency), '.2f')))

PyTorch cpu Inference time = 164.97 ms


## 4. Inference ONNX Model with ONNX Runtime ##

### OpenMP Environment Variable

OpenMP environment variables are very important for CPU inference of Bert model. It has large performance impact on Bert model so you might need set it carefully according to [Performance Test Tool](#Performance-Test-Tool) result in later part of this notebook.

Setting environment variables shall be done before importing onnxruntime. Otherwise, they might not take effect.

In [7]:
import psutil

# You may change the settings in this cell according to Performance Test Tool result.
use_openmp = False

# ATTENTION: these environment variables must be set before importing onnxruntime.
if use_openmp:
    os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
else:
    os.environ["OMP_NUM_THREADS"] = '1'

os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

Now we are ready to inference the model with ONNX Runtime. Here we can see that OnnxRuntime has better performance than PyTorch. 

It is better to use standalone python script like [Performance Test tool](#Performance-Test-tool) to get accurate performance results.

In [8]:
import onnxruntime
import numpy

# Print warning if user uses onnxruntime-gpu instead of onnxruntime package.
if 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
    print("warning: onnxruntime-gpu is not built with OpenMP. You might try onnxruntime package to test CPU inference.")

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time, so it is for debugging only.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_cpu.onnx")
   
if use_openmp:
    sess_options.intra_op_num_threads=1
else:
    sess_options.intra_op_num_threads=psutil.cpu_count(logical=True)

# Specify providers when you use onnxruntime-gpu for CPU inference.
session = onnxruntime.InferenceSession(export_model_path, sess_options, providers=['CPUExecutionProvider'])

latency = []
for i in range(total_samples):
    data = dataset[i]
    # Use contiguous array as input might improve performance.
    # You can check the results from performance test tool to see whether you need it.
    ort_inputs = {
        'input_ids':  numpy.ascontiguousarray(data[0].cpu().reshape(1, max_seq_length).numpy()),
        'input_mask': numpy.ascontiguousarray(data[1].cpu().reshape(1, max_seq_length).numpy()),
        'segment_ids': numpy.ascontiguousarray(data[2].cpu().reshape(1, max_seq_length).numpy())
    }
    start = time.time()
    ort_outputs = session.run(None, ort_inputs)
    latency.append(time.time() - start)
print("OnnxRuntime cpu Inference time = {} ms".format(format(sum(latency) * 1000 / len(latency), '.2f')))

OnnxRuntime cpu Inference time = 91.51 ms


In [9]:
print("***** Verifying correctness *****")
for i in range(2):
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(ort_outputs[i], outputs[i].cpu(), rtol=1e-05, atol=1e-04))

***** Verifying correctness *****
PyTorch and ONNX Runtime output 0 are close: True
PyTorch and ONNX Runtime output 1 are close: True


## 5. Offline Optimization Script and Test Tools

It is recommended to download the [OnnxRuntime Python Tools for BERT](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/bert), and try them on the exported ONNX models. It could help verify whether the model is fully optimized, and get performance test results.

### Download OnnxRuntime Python Tools for Bert
You may copy the whole [directory](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/bert) to a sub-directory named bert_scripts for this notebook. The list of script files might need update if import error happens when you run some script.

In [10]:
import os
import wget

url_prfix = "https://raw.githubusercontent.com/microsoft/onnxruntime/master/onnxruntime/python/tools/bert/"
script_files = ['bert_perf_test.py', 'bert_test_data.py', 'compare_bert_results.py', 'BertOnnxModel.py', 'BertOnnxModelKeras.py', 'BertOnnxModelTF.py', 'Gpt2OnnxModel.py', 'OnnxModel.py', 'bert_model_optimization.py', 'MachineInfo.py']

script_dir = './bert_scripts'
if not os.path.exists(script_dir):
    os.makedirs(script_dir)

for filename in script_files:
    target_file = os.path.join(script_dir, filename)
    if enable_overwrite and os.path.exists(target_file):
        os.remove(target_file)
    if not os.path.exists(target_file):
        wget.download(url_prfix + filename, target_file)
        print("Downloaded", filename)

100% [..............................................................................] 15310 / 15310Downloaded bert_perf_test.py
100% [................................................................................] 9571 / 9571Downloaded bert_test_data.py
100% [................................................................................] 7272 / 7272Downloaded compare_bert_results.py
100% [..............................................................................] 44905 / 44905Downloaded BertOnnxModel.py
100% [..............................................................................] 21565 / 21565Downloaded BertOnnxModelKeras.py
100% [..............................................................................] 26114 / 26114Downloaded BertOnnxModelTF.py
100% [..............................................................................] 22773 / 22773Downloaded OnnxModel.py
100% [................................................................................] 7917 / 7917

#### BERT Optimization Script

Although OnnxRuntime could optimize Bert model exported by PyTorch. Sometime, model cannot be fully optimized due to different reasons:
* A new subgraph pattern is generated by new version of export tool, and the pattern is not covered by older version of OnnxRuntime. For example, Gelu from PyTorch 1.4 is not fused by OnnxRuntime 1.1.2.
* The exported model uses dynamic axis and this makes it harder for shape inference of the graph. That blocks some optimization to be applied.
* Some optimization is better to be done offline. Like change input tensor type from int64 to int32 to avoid extra Cast nodes, or convert model to float16 to achieve better performance in V100 or T4 GPU.

We have python script **bert_model_optimization.py**, which is more flexible in graph pattern matching and model conversion (like float32 to float16). You can also use it to verify whether a Bert model is fully optimized.

In this example, we can see that it introduces optimization that is not provided by onnxruntime: SkipLayerNormalization and bias fusion, which is not fused in OnnxRuntime due to shape inference as mentioned.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

In [11]:
optimized_model_path = './onnx/bert-base-cased-squad_opt_cpu.onnx'
%run ./bert_scripts/bert_model_optimization.py --input $export_model_path --output $optimized_model_path

bert_model_optimization.py: Save optimized model by onnxruntime to ./onnx\bert-base-cased-squad_ort_cpu.onnx
bert_model_optimization.py: Use OnnxRuntime to optimize and save the optimized model to ./onnx\bert-base-cased-squad_ort_cpu.onnx
    BertOnnxModel.py: Fused LayerNormalization count: 0
    BertOnnxModel.py: Fused Reshape count:0
    BertOnnxModel.py: Fused SkipLayerNormalization count: 24
    BertOnnxModel.py: Fused Attention count:0
    BertOnnxModel.py: skip embed layer fusion since mask input is not found
    BertOnnxModel.py: Fused SkipLayerNormalization with Bias count:24
    BertOnnxModel.py: opset verion: 11
        OnnxModel.py: Output model to ./onnx/bert-base-cased-squad_opt_cpu.onnx
    BertOnnxModel.py: EmbedLayer=1, Attention=12, Gelu=12, LayerNormalization=24, Succesful=True
bert_model_optimization.py: The output model is fully optimized.


#### Optimized Graph
When you can open the optimized model using Netron to visualize, the graph is like the following:
<img src='images/optimized_bert_gpu.png'>

For CPU, optimized graph is slightly different: FastGelu is replaced by BiasGelu.

In [12]:
import netron

# Change it to True if want to view the optimized model in browser.
enable_netron = False
if enable_netron:
    # If you encounter error "access a socket in a way forbidden by its access permissions", install Netron as standalone application instead.
    netron.start(optimized_model_path)

#### Model Results Comparison Tool

If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare results from both the original and optimized models. If outputs are all close, it is safe to use the optimized model.

Example of verifying models:

In [13]:
%run ./bert_scripts/compare_bert_results.py --baseline_model $export_model_path --optimized_model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100

100% passed for 100 random inputs given thresholds (rtol=0.001, atol=0.0001).
maximum absolute difference=5.0961971282958984e-06
maximum relative difference=0.003811897709965706


#### Performance Test Tool

This tool measures performance of BERT model inference using OnnxRuntime Python API.

The following command will create 100 samples of batch_size 1 and sequence length 128 to run inference, then calculate performance numbers like average latency and throughput etc. You can increase number of samples (recommended 1000) to get more stable result.

In [14]:
%run ./bert_scripts/bert_perf_test.py --model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100 --test_times 1 --inclusive --all

Generating 100 samples for batch_size=1 sequence_length=128
Extra latency for converting inputs to contiguous: 0.00 ms
Test summary is saved to onnx\perf_results_CPU_B1_S128_20200313-001048.txt


Let's load the summary file and take a look.

In [15]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print(latest_result_file)
print("The best setting is: {} openmp; {} contiguous array".format('use' if result_data['intra_op_num_threads'].iloc[0] == 1 else 'NO', 'use' if result_data['contiguous'].iloc[0] else 'NO'))
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu', 'warmup']
# Hide some latency percentile columns to fit screen width.
columns_to_remove.extend(['Latency_P50', 'Latency_P95'])
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

./onnx\perf_results_CPU_B1_S128_20200313-001048.txt
The best setting is: NO openmp; use contiguous array


,Latency(ms),Latency_P75,Latency_P90,Latency_P99,Throughput(QPS),intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous
0,74.68,76.74,83.75,89.26,13.39,12,1,PASSIVE,True
1,74.70,76.57,85.86,89.30,13.39,12,1,ACTIVE,True
2,75.23,78.13,86.49,88.99,13.29,12,1,ACTIVE,False
3,75.66,78.36,87.60,95.17,13.22,12,1,PASSIVE,False
4,76.56,78.73,91.02,101.58,13.06,1,12,PASSIVE,False
5,76.71,78.99,91.71,98.70,13.04,1,12,PASSIVE,True
6,79.08,80.87,85.98,121.36,12.65,1,12,ACTIVE,True
7,80.30,82.26,96.97,122.55,12.45,1,,,True
8,88.43,91.29,111.84,119.99,11.31,1,6,PASSIVE,False
9,89.03,92.76,110.57,121.06,11.23,1,6,PASSIVE,True


## 6. Additional Info

Note that running Jupyter Notebook has slight impact on performance result since Jupyter Notebook is using system resources like CPU and memory etc. It is recommended to close Jupyter Notebook and other applications, then run the performance test tool in a console to get more accurate performance numbers.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. The machine has GPU but not used in CPU inference.
You might get slower or faster result based on your hardware.

In [16]:
%run ./bert_scripts/MachineInfo.py --silent

{
  "gpu": {
    "driver_version": "441.22",
    "devices": [
      {
        "memory_total": 8589934592,
        "memory_available": 6741864448,
        "name": "GeForce GTX 1070"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz",
    "cores": 6,
    "logical_cores": 12,
    "hz": "3.1920 GHz",
    "l2_cache": "1536 KB",
    "l3_cache": "12288 KB",
    "processor": "Intel64 Family 6 Model 158 Stepping 10, GenuineIntel"
  },
  "memory": {
    "total": 16971259904,
    "available": 2581991424
  },
  "python": "3.6.10.final.0 (64 bit)",
  "os": "Windows-10-10.0.18362-SP0",
  "onnxruntime": {
    "version": "1.2.0",
    "support_gpu": false
  },
  "pytorch": {
    "version": "1.4.0+cpu",
    "support_gpu": false
  },
  "tensorflow": {
    "version": "2.1.0",
    "git_version": "v2.1.0-rc2-17-ge5bf8de410",
    "support_gpu": true
  }
}
